# Import libraries

In [1]:
import pandas as pd
import os
import re
# import ipywidgets as widgets
# from IPython.display import display
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# Due to version of jupyter notebook, I need to enable plotly like so... 

import plotly.io as pio
pio.renderers.default = "notebook_connected"

# Dataset 1

In [3]:
# Load in dataset

df = pd.read_csv('Data/GI_age.csv')

In [4]:
# Brief glimpse of data structure

df.head(10)

,England and Wales Code,England and Wales,Gender identity (7 categories) Code,Gender identity (7 categories),Age (6 categories) Code,Age (6 categories),Observation
0,K04000001,England and Wales,-8,Does not apply,1,Aged 15 years and under,0
1,K04000001,England and Wales,-8,Does not apply,2,Aged 16 to 24 years,0
2,K04000001,England and Wales,-8,Does not apply,3,Aged 25 to 34 years,0
3,K04000001,England and Wales,-8,Does not apply,4,Aged 35 to 49 years,0
4,K04000001,England and Wales,-8,Does not apply,5,Aged 50 to 64 years,0
5,K04000001,England and Wales,-8,Does not apply,6,Aged 65 years and over,0
6,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,1,Aged 15 years and under,0
7,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,2,Aged 16 to 24 years,5809658
8,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,3,Aged 25 to 34 years,7518377
9,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,4,Aged 35 to 49 years,10829667


In [5]:
# Use .shape to return number of (rows, columns)

df.shape

(42, 7)

## Data cleaning

In [6]:
# Finds all substrings which match the regular expression and replaces them with empty string
# string - just another word for text
# substring - part of a string

df.columns = [re.sub(r"\s*\([^)]*\)", "", col) for col in df.columns]

In [7]:
# Lowercase all text and replace empty text with a "_"

df.columns = [col.lower().replace(" ", "_") for col in df.columns]

In [8]:
df.head()

,england_and_wales_code,england_and_wales,gender_identity_code,gender_identity,age_code,age,observation
0,K04000001,England and Wales,-8,Does not apply,1,Aged 15 years and under,0
1,K04000001,England and Wales,-8,Does not apply,2,Aged 16 to 24 years,0
2,K04000001,England and Wales,-8,Does not apply,3,Aged 25 to 34 years,0
3,K04000001,England and Wales,-8,Does not apply,4,Aged 35 to 49 years,0
4,K04000001,England and Wales,-8,Does not apply,5,Aged 50 to 64 years,0


In [9]:
# Get rid of columns with 0 observations

df = df[df.gender_identity_code != -8].reset_index(drop = True) 

In [10]:
df.head(10)

,england_and_wales_code,england_and_wales,gender_identity_code,gender_identity,age_code,age,observation
0,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,1,Aged 15 years and under,0
1,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,2,Aged 16 to 24 years,5809658
2,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,3,Aged 25 to 34 years,7518377
3,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,4,Aged 35 to 49 years,10829667
4,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,5,Aged 50 to 64 years,10966023
5,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,6,Aged 65 years and over,10265910
6,K04000001,England and Wales,2,Gender identity different from sex registered ...,1,Aged 15 years and under,0
7,K04000001,England and Wales,2,Gender identity different from sex registered ...,2,Aged 16 to 24 years,16590
8,K04000001,England and Wales,2,Gender identity different from sex registered ...,3,Aged 25 to 34 years,28375
9,K04000001,England and Wales,2,Gender identity different from sex registered ...,4,Aged 35 to 49 years,38280


In [11]:
df = df[df.age_code != 1] 

In [12]:
# Chain str.replace() calls together to apply multiple string replacements in succession
# Each .replace() call is applied to the result of the previous one
# Assign new column values back to the age column

df['age'] = df['age'].str.replace('Aged ', '').str.replace('to', '-').str.replace(' years', '').str.replace(' and over', '+')


In [13]:
df.head()

,england_and_wales_code,england_and_wales,gender_identity_code,gender_identity,age_code,age,observation
1,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,2,16 - 24,5809658
2,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,3,25 - 34,7518377
3,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,4,35 - 49,10829667
4,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,5,50 - 64,10966023
5,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,6,65+,10265910


## Question

How is gender identity distributed among different age groups?

Some subquestions that this can help us answer:

* What % of trans men are aged 16-24 years?
* Are older age groups overrepresented in the 'non-response' category?

## Data pre-processing

### Calculate percentages

In [14]:
df['percentage'] = ''

for i in df.gender_identity.unique():
    sub = df[df.gender_identity == i]
    perc = sub.observation / sub.observation.sum() * 100
    df.loc[sub.index, 'percentage'] = round(perc,2)

In [15]:
df.head(10)

,england_and_wales_code,england_and_wales,gender_identity_code,gender_identity,age_code,age,observation,percentage
1,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,2,16 - 24,5809658,12.8
2,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,3,25 - 34,7518377,16.56
3,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,4,35 - 49,10829667,23.86
4,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,5,50 - 64,10966023,24.16
5,K04000001,England and Wales,1,Gender identity the same as sex registered at ...,6,65+,10265910,22.62
7,K04000001,England and Wales,2,Gender identity different from sex registered ...,2,16 - 24,16590,14.09
8,K04000001,England and Wales,2,Gender identity different from sex registered ...,3,25 - 34,28375,24.09
9,K04000001,England and Wales,2,Gender identity different from sex registered ...,4,35 - 49,38280,32.5
10,K04000001,England and Wales,2,Gender identity different from sex registered ...,5,50 - 64,21678,18.41
11,K04000001,England and Wales,2,Gender identity different from sex registered ...,6,65+,12852,10.91


# Interactive grouped bar chart + stacked bar chart

## Tooltips

When using different Python libraries geared towards interactive visualisations, you'll often come across 'tooltips'. These are small boxes that provide information when a user hovers over a part of a data visualisation such as: a point on a graph, a bar in a bar chart, or a segment in a pie chart. They are used to display additional information about the data point or object, providing more context without cluttering up the chart. In Plotly tooltips are referred to as 'hover_data'.

When creating them you'll often use a dictionary {} with a key:value, with keys representing the column you'd like to show, and the value representing how you'd like the column data to be displayed, or a True/False value to show or hide the column. By default, Plotly will show the x and y axis data, as well as the data that defines the colour of the bars.

In [16]:
# Let's define our hover_data to remove x and y values, and add in observation column

hover_data = {'observation': True,
              'percentage': ':.2f%',
             'age': False,
             'gender_identity': False}

In [17]:


# First let's create our graph

fig = px.bar(data_frame=df, 
             x='gender_identity', 
             y='percentage', 
             color='age', 
             barmode='group', 
             hover_data = hover_data,
             title = 'Distribution of Gender Identity Categories Among Age Groups', 
             width = 1000, 
             height = 600) 


fig.show()


## Styling/adjust graph layout

As you can see our default graph looks a bit meh. The title isn't aligned to the center, the x axis label has an underscore which isn't very appealing, and some of the x axis category labels are wayyy too long. We'll sort this out by using .update_layout() function, which allows us to make changes to our graph once it's been created. To update our xaxes with custom tick labels, we'll use the .update_xaxes function. 


## Interactive legends

By default Plotly's legends are interactive. When you click on a legend category, it hides all data associated with that category. This allows us to easily explore patterns and trends in our data. 


In [18]:
### Use update_layout to adjust the size of the graph, move the legend, and add titles

fig.update_layout(
    title ={'text':'Distribution of Gender Identity Categories Among Age Groups',
            'x': 0.5,
           },
    xaxis_title = 'Gender Identity',
    yaxis_title = 'Percentage',
    width = 1000,
    height = 900,

    # Move the legend below the plot
    legend=dict(
        orientation="v",
        yanchor="top",
        y=-0.3,
        xanchor="center",
        x=1,
        title_text = "Age"
    ))

fig.update_xaxes(tickvals = df['gender_identity'].unique(),
                 ticktext = ["Cisgender",
                             "Gender identity different from sex", 
                             "Trans woman",
                             "Trans man",
                             "All other identities","Not answered"],
                title_standoff = 40)
                        

## Further styling options..

In [19]:

# Update the layout to make the graph look less basic
# fig.update_layout(
#     plot_bgcolor='rgba(0,0,0,0)',  # Transparent background within the plot
#     paper_bgcolor='rgba(0,0,0,0)',  # Transparent background around the plot
#     title={
#         'text': 'Distribution of Gender Identity Categories Among Age Groups',
#         'y':0.92,
#         'x':0.5,
#         'xanchor': 'center',
#         'yanchor': 'top'},
    
#     title_font=dict(size=25, color='navy'),
#     font=dict(family='Arial, sans-serif', size=18, color='RebeccaPurple'),
#     xaxis=dict(tickangle=-45, title_standoff=25),
#     xaxis_title_font=dict(size=20, family='Courier', color='black'),
#     yaxis_title_font=dict(size=20, family='Courier', color='black'),
#     legend=dict(
#         title_text='Age Groups',
#         bgcolor='rgba(255,255,255,0.5)',
#         bordercolor='Black',
#         borderwidth=2
#     ),
#     showlegend=True
# )

# # Style the bars
# fig.update_traces(
#     marker=dict(line=dict(width=1.5, color='DarkSlateGrey')),
#     opacity=0.8
# )

# fig.show()


In [20]:
import plotly.express as px

fig = px.bar(data_frame=df, x='gender_identity', y='percentage', color='age', hover_data = ['observation'])
fig.update_layout(
    title ={'text':'Distribution of Gender Identity Categories Among Age Groups',
            'x': 0.5,
           },
    xaxis_title = 'Gender Identity',
    yaxis_title = 'Percentage',
    # Set the size of the figure
    width=1000,  # Adjust the width to your preference
    height=600,  # Adjust the height to your preference
    # Move the legend below the plot
    legend=dict(
        orientation="v",
        yanchor="bottom",
        y=-0.5,
        xanchor="center",
        x=1.1
    ))

fig.update_xaxes(tickvals = df['gender_identity'].unique(),
                 ticktext = ["Cisgender",
                             "Gender identity different from sex", 
                             "Trans woman",
                             "Trans man",
                             "All other identities","Not answered"],
                title_standoff = 40)
                        
fig.show()


# Dataset 2

In [21]:
# Load in dataset

df2 = pd.read_csv('Data/GI_ethnic.csv')

In [22]:
# Brief glimpse at underlying data structure

df2.head(10)

,Lower tier local authorities Code,Lower tier local authorities,Gender identity (4 categories) Code,Gender identity (4 categories),Ethnic group (8 categories) Code,Ethnic group (8 categories),Observation
0,E06000001,Hartlepool,-8,Does not apply,-8,Does not apply,0
1,E06000001,Hartlepool,-8,Does not apply,1,"Asian, Asian British or Asian Welsh",0
2,E06000001,Hartlepool,-8,Does not apply,2,"Black, Black British, Black Welsh, Caribbean o...",0
3,E06000001,Hartlepool,-8,Does not apply,3,Mixed or Multiple ethnic groups,0
4,E06000001,Hartlepool,-8,Does not apply,4,"White: English, Welsh, Scottish, Northern Iris...",0
5,E06000001,Hartlepool,-8,Does not apply,5,White: Irish,0
6,E06000001,Hartlepool,-8,Does not apply,6,"White: Gypsy or Irish Traveller, Roma or Other...",0
7,E06000001,Hartlepool,-8,Does not apply,7,Other ethnic group,0
8,E06000001,Hartlepool,1,Gender identity the same as sex registered at ...,-8,Does not apply,0
9,E06000001,Hartlepool,1,Gender identity the same as sex registered at ...,1,"Asian, Asian British or Asian Welsh",1008


In [23]:
df2.shape

(10592, 7)

## Data Cleaning

In [24]:
# Finds all substrings which match the regular expression and replaces them with empty string
# string - just another word for text
# substring - part of a string

df2.columns = [re.sub(r"\s*\([^)]*\)", "", col) for col in df2.columns]

In [25]:
# Lowercase all text and replace empty text with a "_"

df2.columns = [col.lower().replace(" ", "_") for col in df2.columns]

In [26]:
# Let's shorten the local authority column names as they are wayyyy too long

df2.rename(columns={'lower_tier_local_authorities_code':'LA_code', 'lower_tier_local_authorities': 'LA_name'}, inplace=True)

In [27]:
df2

,LA_code,LA_name,gender_identity_code,gender_identity,ethnic_group_code,ethnic_group,observation
0,E06000001,Hartlepool,-8,Does not apply,-8,Does not apply,0
1,E06000001,Hartlepool,-8,Does not apply,1,"Asian, Asian British or Asian Welsh",0
2,E06000001,Hartlepool,-8,Does not apply,2,"Black, Black British, Black Welsh, Caribbean o...",0
3,E06000001,Hartlepool,-8,Does not apply,3,Mixed or Multiple ethnic groups,0
4,E06000001,Hartlepool,-8,Does not apply,4,"White: English, Welsh, Scottish, Northern Iris...",0
...,...,...,...,...,...,...,...
10587,W06000024,Merthyr Tydfil,3,Not answered,3,Mixed or Multiple ethnic groups,28
10588,W06000024,Merthyr Tydfil,3,Not answered,4,"White: English, Welsh, Scottish, Northern Iris...",2258
10589,W06000024,Merthyr Tydfil,3,Not answered,5,White: Irish,5
10590,W06000024,Merthyr Tydfil,3,Not answered,6,"White: Gypsy or Irish Traveller, Roma or Other...",167


In [28]:
# Again we have redundant categories...
# Let's get rid of 'Does not apply' for the gender identity and ethnic group column

df2 = df2[(df2.gender_identity_code != -8) & (df2.ethnic_group_code != -8)].reset_index(drop = True) 

In [29]:
df2.head(30)

,LA_code,LA_name,gender_identity_code,gender_identity,ethnic_group_code,ethnic_group,observation
0,E06000001,Hartlepool,1,Gender identity the same as sex registered at ...,1,"Asian, Asian British or Asian Welsh",1008
1,E06000001,Hartlepool,1,Gender identity the same as sex registered at ...,2,"Black, Black British, Black Welsh, Caribbean o...",253
2,E06000001,Hartlepool,1,Gender identity the same as sex registered at ...,3,Mixed or Multiple ethnic groups,335
3,E06000001,Hartlepool,1,Gender identity the same as sex registered at ...,4,"White: English, Welsh, Scottish, Northern Iris...",67682
4,E06000001,Hartlepool,1,Gender identity the same as sex registered at ...,5,White: Irish,151
5,E06000001,Hartlepool,1,Gender identity the same as sex registered at ...,6,"White: Gypsy or Irish Traveller, Roma or Other...",825
6,E06000001,Hartlepool,1,Gender identity the same as sex registered at ...,7,Other ethnic group,334
7,E06000001,Hartlepool,2,Gender identity different from sex registered ...,1,"Asian, Asian British or Asian Welsh",21
8,E06000001,Hartlepool,2,Gender identity different from sex registered ...,2,"Black, Black British, Black Welsh, Caribbean o...",8
9,E06000001,Hartlepool,2,Gender identity different from sex registered ...,3,Mixed or Multiple ethnic groups,7


## Data pre-processing

## Calculate % of each ethnic group in each LA

In [30]:
# First we're going to group our data by LA_name, Ethnic group, and Observation and sum our observations 
# Leaves us with the total of each ethnic group in each local authority

ethnic_totals = df2.groupby(['LA_name', 'ethnic_group'])['observation'].sum().reset_index(name='Ethnic_sum')

In [31]:
ethnic_totals

,LA_name,ethnic_group,Ethnic_sum
0,Adur,"Asian, Asian British or Asian Welsh",1098
1,Adur,"Black, Black British, Black Welsh, Caribbean o...",437
2,Adur,Mixed or Multiple ethnic groups,898
3,Adur,Other ethnic group,563
4,Adur,"White: English, Welsh, Scottish, Northern Iris...",47460
...,...,...,...
2312,York,Mixed or Multiple ethnic groups,2412
2313,York,Other ethnic group,1574
2314,York,"White: English, Welsh, Scottish, Northern Iris...",150522
2315,York,"White: Gypsy or Irish Traveller, Roma or Other...",8571


In [32]:
# Step 3: Calculate total observations for each local authority
la_totals = df2.groupby('LA_name')['observation'].sum().reset_index(name='LA_sum')

la_totals

,LA_name,LA_sum
0,Adur,52951
1,Allerdale,80520
2,Amber Valley,105061
3,Arun,139550
4,Ashfield,102847
...,...,...
326,Wrexham,110274
327,Wychavon,110716
328,Wyre,94353
329,Wyre Forest,84562


In [33]:
# Merge the summed ethnic group data with the total la observations to calculate percentage of each ethnic group in each LA

merged = pd.merge(ethnic_totals, la_totals, on='LA_name')

In [34]:
merged

,LA_name,ethnic_group,Ethnic_sum,LA_sum
0,Adur,"Asian, Asian British or Asian Welsh",1098,52951
1,Adur,"Black, Black British, Black Welsh, Caribbean o...",437,52951
2,Adur,Mixed or Multiple ethnic groups,898,52951
3,Adur,Other ethnic group,563,52951
4,Adur,"White: English, Welsh, Scottish, Northern Iris...",47460,52951
...,...,...,...,...
2312,York,Mixed or Multiple ethnic groups,2412,172082
2313,York,Other ethnic group,1574,172082
2314,York,"White: English, Welsh, Scottish, Northern Iris...",150522,172082
2315,York,"White: Gypsy or Irish Traveller, Roma or Other...",8571,172082


In [35]:
# Step 4: Calculate the percentage of each ethnic group within each local authority

merged['Percentage'] = (merged['Ethnic_sum'] / merged['LA_sum'] * 100).round(2)


In [36]:
merged.head(10)

,LA_name,ethnic_group,Ethnic_sum,LA_sum,Percentage
0,Adur,"Asian, Asian British or Asian Welsh",1098,52951,2.07
1,Adur,"Black, Black British, Black Welsh, Caribbean o...",437,52951,0.83
2,Adur,Mixed or Multiple ethnic groups,898,52951,1.70
3,Adur,Other ethnic group,563,52951,1.06
4,Adur,"White: English, Welsh, Scottish, Northern Iris...",47460,52951,89.63
5,Adur,"White: Gypsy or Irish Traveller, Roma or Other...",2025,52951,3.82
6,Adur,White: Irish,470,52951,0.89
7,Allerdale,"Asian, Asian British or Asian Welsh",474,80520,0.59
8,Allerdale,"Black, Black British, Black Welsh, Caribbean o...",64,80520,0.08
9,Allerdale,Mixed or Multiple ethnic groups,361,80520,0.45


## Calculate Ethnic Group NR Rates (%'s) Within LAs

In [37]:
# First we're going to group our data by LA_name, Ethnic group, and Observation and sum our observations 
# Leaves us with the total of each ethnic group in each local authority
ethnic_totals = df2.groupby(['LA_name', 'ethnic_group'])['observation'].sum().reset_index(name='Ethnic_sum')

# Calculate total observations for each local authority
la_totals = df2.groupby('LA_name')['observation'].sum().reset_index(name='LA_sum')

# Merge the summed ethnic group data with the total LA observations to calculate the percentage of each ethnic group in each LA
merged = pd.merge(ethnic_totals, la_totals, on='LA_name')
merged['Percentage'] = (merged['Ethnic_sum'] / merged['LA_sum'] * 100).round(2)

# Now calculate the non-response % for each ethnic group in each LA:
# This is the sum of all observations for each ethnic group within each LA
ethnic_group_totals = df2.groupby(['LA_name', 'ethnic_group'])['observation'].sum().reset_index(name='Ethnic_group_total')

# This is the sum of non-responses for each ethnic group within each LA
non_response_totals = df2[df2['gender_identity'] == 'Not answered'].groupby(['LA_name', 'ethnic_group'])['observation'].sum().reset_index(name='Non_response_total')

# Merge the totals with the non-response totals
merged_data = pd.merge(ethnic_group_totals, non_response_totals, on=['LA_name', 'ethnic_group'], how='left')

# Calculate the non-response percentage for each ethnic group within each LA
merged_data['Eth_NR_Perc'] = round((merged_data['Non_response_total'] / merged_data['Ethnic_group_total']) * 100, 2)

# Merge the non-response data with the percentage of each ethnic group within each LA
nr = pd.merge(merged_data, merged[['LA_name', 'ethnic_group', 'Percentage']], on=['LA_name', 'ethnic_group'], how='left')

# This gives you the final dataset with both the non-response rate and the percentage of each ethnic group within each LA
nr = nr.reset_index(drop=True)


In [38]:
nr.head(10)

,LA_name,ethnic_group,Ethnic_group_total,Non_response_total,Eth_NR_Perc,Percentage
0,Adur,"Asian, Asian British or Asian Welsh",1098,61,5.56,2.07
1,Adur,"Black, Black British, Black Welsh, Caribbean o...",437,36,8.24,0.83
2,Adur,Mixed or Multiple ethnic groups,898,46,5.12,1.70
3,Adur,Other ethnic group,563,29,5.15,1.06
4,Adur,"White: English, Welsh, Scottish, Northern Iris...",47460,2131,4.49,89.63
5,Adur,"White: Gypsy or Irish Traveller, Roma or Other...",2025,150,7.41,3.82
6,Adur,White: Irish,470,22,4.68,0.89
7,Allerdale,"Asian, Asian British or Asian Welsh",474,44,9.28,0.59
8,Allerdale,"Black, Black British, Black Welsh, Caribbean o...",64,4,6.25,0.08
9,Allerdale,Mixed or Multiple ethnic groups,361,23,6.37,0.45


# Interactive scatterplot

In this section we're going to:

1. **Create a simple scatterplot** exploring the relationship between the percentage of asian citizens within local authorities and their non-response rates 

2. Implement a widget and **update our scatterplot with a 'python callback'**


In [39]:
# Subset dataframe so that we only have responses from the asian ethnic group
# Remember to reset_index() so our rows are in order

asian = nr[nr.ethnic_group == 'Asian, Asian British or Asian Welsh'].reset_index(drop = True)

In [40]:
# Check it out..

asian.head()

,LA_name,ethnic_group,Ethnic_group_total,Non_response_total,Eth_NR_Perc,Percentage
0,Adur,"Asian, Asian British or Asian Welsh",1098,61,5.56,2.07
1,Allerdale,"Asian, Asian British or Asian Welsh",474,44,9.28,0.59
2,Amber Valley,"Asian, Asian British or Asian Welsh",977,67,6.86,0.93
3,Arun,"Asian, Asian British or Asian Welsh",2117,162,7.65,1.52
4,Ashfield,"Asian, Asian British or Asian Welsh",1502,129,8.59,1.46


In [41]:
# Let's define our hover_data to remove x and y values, and add in some columns

hover_data = {'LA_name': True,
            'Percentage': ':.2f',
              'Eth_NR_Perc': ':.2f%',
             'Non_response_total': True,
             'Ethnic_group_total': True}

In [42]:
fig = px.scatter(asian, x = "Percentage", y = "Eth_NR_Perc", hover_data = hover_data, trendline = "ols")


fig.update_layout(
    title ={'text':'Non-Response Rates of the Asian Ethnic Group Across Local Authorities',
            'x': 0.5,
           },
    xaxis_title = 'Percentage of ethnic group',
    yaxis_title = 'Non-response rate',
    width = 900,
    height = 900,)

# Set marker size
fig.update_traces(marker_size=7)
fig.show()

## Dropdown selection

What we're going to do now, is use Plotly's 'updatemenus' in conjunction with the 'update' method to create a dropdown where we can switch between the Asian ethnic group, and White.

### Step 1: Initialise figure and add traces

We'll start by creating a graph_object figure. We use graph objects here instead of Plotly express's scatter method, because graph objects offer more control over how plots are constructed. It allows us to add 'traces', which refer to a set of data. In our example, we want to add a trace with the data points relating to our asian ethnic group, and another one for our white ethnic group. This will start to make sense when we look at the code below. 

In [43]:
# Initialize figure
fig = go.Figure()


# Let's take a look..
# This is our building block
fig.show()

In [44]:
# Add trace for Asian ethnic group

fig.add_trace(
#     specify graph type - in our case, scatterplot
    go.Scatter(
#         specify x-axis values - subset our dataframe by ethnic group and select Percentage column
        x=nr[nr['ethnic_group'] == 'Asian, Asian British or Asian Welsh']['Percentage'],
#         specify y-axis values
        y=nr[nr['ethnic_group'] == 'Asian, Asian British or Asian Welsh']['Eth_NR_Perc'],
#         we want marker points for our dps, not lines which is the default
        mode = 'markers',
#         we want the Asian ethnic group graph to be the default
        visible=True 
    )
)

# Add trace for the White ethnic group

fig.add_trace(
    go.Scatter(
        x=nr[nr['ethnic_group'] == 'White: English, Welsh, Scottish, Northern Irish or British']['Percentage'],
        y=nr[nr['ethnic_group'] == 'White: English, Welsh, Scottish, Northern Irish or British']['Eth_NR_Perc'],
        name='White',
        mode = 'markers',
#         we want to hide this ethnic group by default, as we want to use dropdown to switch 
        visible=False  
    )
)

# Add dropdown

fig.update_layout(
    updatemenus=[
#         Inside updatemenus, the dict defines the configuration of a single dropdown menu
        dict(
#             We have a list of dictionaries called 'buttons', and each dictionary represents a button...
#             in the dropdown menu
            buttons=list([
                dict(label="Asian",
#                      specifies action to be performed when button is clicked...
#                      i.e. it will update the graph with the following arguments
                     method="update",
#                      Specify changes, i.e. determines which traces are visible
                     args=[{"visible": [True, False]},
                           {"title": "Non-Response Rates of the Asian Ethnic Group Across Local Authorities"}]),
                dict(label="White",
                     method="update",
                     args=[{"visible": [False, True]},
                           {"title": "Non-Response Rates of the White Ethnic Group Across Local Authorities"}]),
            ]),
#             Specifies which direction the menu opens
            direction="down",
#             Highlights the button that is currently active
            showactive=True,
        )
    ]
)

# Set default title and layout properties
fig.update_layout(
    title_text="Non-Response Rates of the Asian Ethnic Group Across Local Authorities",
    xaxis_title="Percentage of Ethnic Group",
    yaxis_title="Non-response Rate",
)

# Show figure
fig.show()

# Dataset 3

In [ ]:
df3 = pd.read_csv('Data/GI_occ.csv')

In [ ]:
df3

In [ ]:
df3['Occupancy rating for rooms (6 categories)'].unique()

## Data cleaning

In [ ]:
# Remove the prefix 'Occupancy rating of rooms: ' from each value in the column
df3['Occupancy rating for rooms (6 categories)'] = df3['Occupancy rating for rooms (6 categories)'].str.replace('Occupancy rating of rooms: ', '')

In [ ]:
# Finds all substrings which match the regular expression and replaces them with empty string
# string - just another word for text
# substring - part of a string

df3.columns = [re.sub(r"\s*\([^)]*\)", "", col) for col in df3.columns]

In [ ]:
# Lowercase all text and replace empty text with a "_"

df3.columns = [col.lower().replace(" ", "_") for col in df3.columns]

In [ ]:
# Let's shorten the local authority column names as they are wayyyy too long

df3.rename(columns={'lower_tier_local_authorities_code':'LA_code', 'lower_tier_local_authorities': 'LA_name'}, inplace=True)

In [ ]:
# Again we have redundant categories...
# Let's get rid of 'Does not apply' for the gender identity and ethnic group column

df3 = df3[(df3.gender_identity_code != -8) & (df3.occupancy_rating_for_rooms_code != -8)].reset_index(drop = True) 

## Data pre-processing

In [ ]:
df3['percentage'] = ''

p_list = []
pp_list = []

for x in df3.LA_name.unique():
    sub = df3[df3.LA_name == x]
    p_list = []
    for y in df3.gender_identity.unique():
        sub2 = sub[sub.gender_identity == y]
        perc = (sub2.observation / sub.observation.sum() * 100).round(2)
        p_list.append(perc.tolist())
    pp_list += p_list

pp_list = [item for sublist in pp_list for item in sublist]

df3['percentage'] = pp_list

In [ ]:
df3.head(50)

In [ ]:
# We will group by LA name and observation, then sum the counts
# Column name for this new df will be 'Total_Observation'
total_population = df3.groupby('LA_name')['observation'].sum().reset_index(name='Total_Observation')
# Create a dataframe to store the results
final_df = total_population.copy()
final_df
# Iterate through each unique religion category
for i in df3['occupancy_rating_for_rooms'].unique():
    
#     Subset dataframe by that religious category and assign results to new df
    spec = df3[df3.occupancy_rating_for_rooms == i]


#     Create a new df which holds the total observations for that religious category
# ..within each LA
    spec_tot = spec.groupby('LA_name')['observation'].sum().reset_index(name=f'{i}_Observation')
#     Then calculate the % of each religious group by dividing category observation by total LA population
    spec_tot[f'{i}_Percentage'] = (spec_tot[f'{i}_Observation'] / total_population['Total_Observation'] * 100).round(2)
#     Merge df where we will store results with spec_tot 
    final_df = pd.merge(final_df, spec_tot[['LA_name', f'{i}_Percentage', f'{i}_Observation']])

In [ ]:
	
# Start by creating a new df which subsets our original df according to our condition

non_response_data = df3[df3.gender_identity == 'Trans man']

In [ ]:
# Calculate total non-response counts for each religious category by grouping observations

non_resp_totals = non_response_data.groupby(['LA_name', 'occupancy_rating_for_rooms'])['observation'].sum().reset_index(name='TW_Observation')


In [ ]:
# Let's check it out...

non_resp_totals.head()

In [ ]:
# Create a copy of non_resp_totals to store the results
non_resp_results = non_resp_totals.copy()

# Merge the Non_Response_Observation and Total_Observation columns
merged_df = pd.merge(non_resp_totals, final_df[['LA_name', 'Total_Observation']], on='LA_name', how='left')

In [ ]:
# Calculate the non-response percentage for each religion category

merged_df['TW_Percentage'] = (merged_df['TW_Observation'] / merged_df['Total_Observation'] * 100).round(2)

In [ ]:
merged_df.head()

In [ ]:
# Pivot the dataframe to create the columns for each religion category

pivot_df = pd.pivot_table(merged_df, values='TW_Percentage', index='LA_name', columns='occupancy_rating_for_rooms', fill_value=0)
pivot_df.head()

In [ ]:
pivot_df.columns = [f'{col}_TW' for col in pivot_df.columns]

In [ ]:
# Merge the pivot_df with the final_df, which we created in phase 1
final_df = pd.merge(final_df, pivot_df.reset_index(), on='LA_name', how='left')

In [ ]:
final_df.head()

In [ ]:
final_df['+1_TW'].value_counts()

In [ ]:
# used to create visualisations
import matplotlib.pyplot as plt

plt.scatter(final_df['-2 or less_Percentage'], final_df['-2 or less_TW'])

In [ ]:
# Suppose you want to rename 'Trans man' and 'Trans woman' entries to 'Transgender'
df3['gender_identity'] = df3['gender_identity'].replace({'Gender identity different from sex registered at birth but no specific identity given': 'Gender identity different from sex registered at birth'})

In [ ]:
plus2 = df3[df3.occupancy_rating_for_rooms == "+2 or more"]

In [ ]:
df3.gender_identity.unique()

### Merged with second dataset

In [ ]:
# Merge the datasets on 'LA_code' and 'gender_identity_category'
merged_df = pd.merge(
    white[['LA_name', 'gender_identity', 'percentage']],  # Make sure to select the correct column for white percentage
    plus2[['LA_name', 'gender_identity', 'percentage']],  # Make sure to select the correct column for +2 occupancy rating
    on=['LA_name', 'gender_identity'],
    suffixes=('_ethnic', '_occup'))

In [ ]:
merged_df.head(10)

In [ ]:
hart = df3[df3.LA_name == 'Hartlepool']
hart.observation.sum()